# Using SQL in Python

In this notebook we'll practice using SQL to extract and transform some US State population data.

We'll use pandasql to execute SQL on pandas dataframes.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandasql import PandaSQL

%matplotlib inline

## Set up pysqldf

In [2]:
# Setting up an instance of PandaSQL to pass SQL commands to
pysqldf = PandaSQL()

## Load Data

In [3]:
# Load state population data
state_population = pd.read_csv('../data/state-population.csv')
state_population = state_population.rename({'state/region':'abbreviation'},
                                           axis=1)

# Load state area data
state_areas = pd.read_csv('../data/state-areas.csv')
state_areas = state_areas.rename({'area (sq. mi)':'area'},axis=1)

# Load state abbreviation data
state_abbreviations = pd.read_csv('../data/state-abbrevs.csv')

In [4]:
# look at each data's column namess
print("------state_population------")
display(state_population.head())
print("------state_areas------")
display(state_areas.head())
print("------state_abbreviations------")
display(state_abbreviations.head())

------state_population------


,abbreviation,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


------state_areas------


,state,area
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


------state_abbreviations------


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


## Practice SQL

In [5]:
# Write SQL to print out:
#    all columns from table state_areas limited to the first 3 rows 
#    aka state_areas.head(3)
sql = """
SELECT *
FROM state_areas
LIMIT 3
"""
pysqldf(sql)

,state,area
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006


In [6]:
# Write SQL to print out the same dataframe as given by this pandas call:
#    state_population.loc[:,['abbreviation']].iloc[:3]
sql = """
SELECT abbrevIation
FROM state_population
LIMIT 3
"""
pysqldf(sql)

,abbreviation
0,AL
1,AL
2,AL


In [7]:
# check the output is aligned with the pandas call
state_population.loc[:,['abbreviation']].iloc[:3]

,abbreviation
0,AL
1,AL
2,AL


---

In [8]:
# Write SQL to print out:
#    columns state and area from table state_areas for rows with state starting with 'Mi'
sql = """
SELECT *
FROM state_areas
WHERE state LIKE 'Mi%'
"""
pysqldf(sql)

,state,area
0,Michigan,96810
1,Minnesota,86943
2,Mississippi,48434
3,Missouri,69709


In [9]:
# Write SQL to print out:
#    columns state and area from table state_areas 
#    for rows with state starting with 'Mi' and area greater than 80000
sql = """
SELECT *
FROM state_areas
WHERE state LIKE 'Mi%' AND area > 80000
"""
pysqldf(sql)

,state,area
0,Michigan,96810
1,Minnesota,86943


In [10]:
# Write SQL to print out:
#    all columns from table state_areas 
#    LEFT JOINed with state_abbreviations ON state
#    limited to the first 3 rows
sql = """
SELECT ar.state, area, abbreviation
FROM state_areas ar
LEFT JOIN state_abbreviations ab ON ar.state = ab.state
LIMIT 3
"""
pysqldf(sql)

,state,area,abbreviation
0,Alabama,52423,AL
1,Alaska,656425,AK
2,Arizona,114006,AZ


In [11]:
# Write SQL to print out:
#    all columns from table state_areas aliased as s_area
#    INNER JOINed with state_abbreviations aliased as s_abb ON state
#    INNER JOINed with state_population aliased as s_pop ON abbreviation
#    limited to the first 3 rows
sql = """
SELECT *
FROM state_areas s_area
INNER JOIN state_abbreviations s_abb ON s_area.state = s_abb.state
INNER JOIN state_population s_pop ON s_abb.abbreviation = s_pop.abbreviation
LIMIT 3
"""
pysqldf(sql)

,state,area,state,abbreviation,abbreviation,ages,year,population
0,Alabama,52423,Alabama,AL,AL,total,1990,4050055.0
1,Alabama,52423,Alabama,AL,AL,total,1991,4099156.0
2,Alabama,52423,Alabama,AL,AL,total,1992,4154014.0


In [12]:
# Write SQL to print out:
#    s_area.state,
#    s_area.area,
#    s_pop.year,
#    s_pop.population
#    from table state_areas aliased as s_area
#    INNER JOINed with state_abbreviations aliased as s_abb ON state
#    INNER JOINed with state_population aliased as s_pop ON abbreviation
#    where s_pop.ages is 'total'
#    ordered by s_area.state, s_pop.year
sql = """
SELECT
s_area.state,
s_area.area,
s_pop.year,
s_pop.population
FROM state_areas s_area
INNER JOIN state_abbreviations s_abb ON s_area.state = s_abb.state
INNER JOIN state_population s_pop ON s_abb.abbreviation = s_pop.abbreviation
WHERE s_pop.ages = 'total'
ORDER BY s_area.state, s_pop.year ASC
"""
pysqldf(sql)
# you should see 1224 rows and 4 columns

,state,area,year,population
0,Alabama,52423,1990,4050055.0
1,Alabama,52423,1991,4099156.0
2,Alabama,52423,1992,4154014.0
3,Alabama,52423,1993,4214202.0
4,Alabama,52423,1994,4260229.0
...,...,...,...,...
1219,Wyoming,97818,2009,559851.0
1220,Wyoming,97818,2010,564222.0
1221,Wyoming,97818,2011,567329.0
1222,Wyoming,97818,2012,576626.0


In [13]:
# Experiment with additional SQL calls 
# For example, state_population contains more regions than there are states in state_areas
print(f"# of regions in state_population: {len(state_population.abbreviation.unique())}")
print(f"# of regions in state_areas: {len(state_areas.state.unique())}")

#     so different join types (left, right) will give different results

# As an additional challenge:
#  calculate the total average population per state in thousands over the years observed
sql = """
SELECT s_area.state, AVG(population)/1000 as average_population_in_k
FROM state_population s_pop
INNER JOIN state_abbreviations s_abb ON s_abb.abbreviation = s_pop.abbreviation
RIGHT JOIN state_areas s_area ON s_area.state = s_abb.state 
GROUP BY year, s_area.state
"""
# INNER JOIN removes Puerto Rico

pysqldf(sql)

# of regions in state_population: 53
# of regions in state_areas: 52


,state,average_population_in_k
0,Puerto Rico,NaN
1,Alabama,2550.0480
2,Alaska,365.3960
3,Arizona,2345.0685
4,Arkansas,1488.7595
...,...,...
1220,Virginia,5062.4700
1221,Washington,4283.6005
1222,West Virginia,1117.9910
1223,Wisconsin,3525.2445
